Script untuk training model CNN dengan Dataset dan menyimpan model

**Output yang didapatkan:**
*   Model CNN hasil training dalam format .h5
*   Plot akurasi & nilai loss pada data training
*   Plot akurasi & nilai loss pada data validation
*   Akurasi pada data testing

**Prasyarat:**
*   Memiliki dataset yang sudah displit berupa data training, data validation, dan data testing
*   Sudah mengupload data tersebut ke Google Drive

**Cara menggunakan:**
1. Upload script kode ini ke Google Drive, lalu buka menggunakan Google Colab
2. Upload file dataset yang berisi data training, data validation dan data testing ke direktori Google Drive yang diinginkan (format .zip)
3. Ubah variabel `gdrive_zip` sesuai lokasi file zip dataset
4. Sesuaikan nama folder dataset kalian pada variabel `base_dir`, `training_dir`, `validation_dir` dan `test_dir`
5. Atur parameter seperti input size gambar dan epoch
6. Tentukan model yang ingin digunakan pada variabel `base_model`
7. Tentukan lokasi penyimpanan dan penamaan file model (untuk callback `checkpoint_filepath` dan juga hasil training akhir `model.save()`) 
8. Tentukan loaksi penyimpanan dan penamaan file history jika ingin menyimpannya
9. Jalankan cell satu-persatu










In [ ]:
# menghubungkan aplikasi ke Google Drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# memuat dataset
import zipfile,os,shutil

# ATUR DIREKTORI DATASET
gdrive_zip = 'drive/MyDrive/Dataset/EuroSAT 6-2-2.zip'
zip_ref = zipfile.ZipFile(gdrive_zip, 'r') # opens the zip file in read mode
zip_ref.extractall('/tmp') # ekstrak file ke folder /tmp (direktori pada Google Colab)
zip_ref.close()

# ATUR DIREKTORI FOLDER DATA TRAINING DAN VALIDATION SESUAI NAMA FOLDER
base_dir = '/tmp/EuroSAT 6-2-2'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'val')
test_dir = os.path.join(base_dir, 'test')

In [ ]:
# ATUR PARAMETER

#ukuran default model pada umumnya (resnet, vgg16, densenet): 224px
#lebih lengkap untuk dimensi input model dapat dilihat di halaman masing-masing model, https://www.tensorflow.org/api_docs/python/tf/keras/applications
#ukuran asli dataset eurosat: 64px
img_height = 224 # atur dimensi piksel masukan
img_width = img_height
img_channel = 3

batch_size = 64 # jumlah data yang akan diproses dalam satu waktu (pada sekali iterasi training)
epoch = 50  # jumlah iterasi training (pada keseluruhan dataset)

In [ ]:
# image generator

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# jika mau atur data augmentation, bisa ubah parameter ImageGenerator() dibawah, baik untuk data training maupun data testing
train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')
 
val_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

train_generator = train_datagen.flow_from_directory(
        train_dir, 
        target_size=(img_width, img_height), 
        batch_size=batch_size,
        class_mode='categorical')

validation_generator = val_datagen.flow_from_directory(
        validation_dir, 
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical')

In [ ]:
# image generator untuk data testing

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=1,
    class_mode='categorical')


In [ ]:
num_classes = train_generator.num_classes # informasi jumlah kelas
train_img_total = train_generator.samples # informasi jumlah gambar data training
val_img_total = validation_generator.samples # informasi jumlah gambar data validation

print(num_classes, train_img_total, val_img_total)

10 16200 5400


In [ ]:
# tentukan model yang akan digunakan. 
# daftar model yang tersedia dapat dilihat di https://www.tensorflow.org/api_docs/python/tf/keras/applications

#base_model = tf.keras.applications.vgg16.VGG16( # model VGG16
#base_model = tf.keras.applications.densenet.DenseNet121( # model DenseNet121
base_model = tf.keras.applications.resnet50.ResNet50( # model ResNet50                                                 
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(img_width, img_height, img_channel),
    pooling='avg',
)

x = base_model.output
x = tf.keras.layers.Dense(512, activation='relu')(x)
predictions = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=predictions)

model.summary()
model.compile(loss='categorical_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
# callback untuk menyimpan model ketika mendapatkan akurasi yang lebih tinggi
# hal ini penting karena ketika epoch makin tinggi tidak ada jaminan akurasi juga semakin tinggi

from tensorflow.keras.callbacks import ModelCheckpoint

# ATUR DIREKTORI DAN PENAMAAN MODEL
# jika namanya sama, maka model yg lama akan tereplace
checkpoint_filepath = '/content/drive/MyDrive/Landsat/Model/ResNet50 x224.h5'

# dynamic naming
# contoh penamaan berdasarkan epoch keberapa dan nilai lossnya
#checkpoint_filepath = 'weights.{epoch:02d}-{val_loss:.2f}.h5'

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# mulai proses tarining

history = model.fit(
    train_generator,
    steps_per_epoch = train_img_total//batch_size,
    epochs = epoch,
    validation_data = validation_generator,
    validation_steps = val_img_total//batch_size,
    verbose = 2,
    callbacks=[model_checkpoint_callback]
)

In [ ]:
# menampilkan grafik akurasi pada proses training & validation

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Accuracy Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="lower right") # atur posisi label
plt.show()

In [ ]:
# menampilkan grafik nilai loss pada proses training dan validation

from matplotlib import pyplot as plt

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Loss Plot')
plt.ylabel('Value')
plt.xlabel('Epoch')
plt.legend(loc="upper right") # atur posisi label
plt.show()

In [ ]:
# testing model terhadap data testing dengan menghitung akurasi dan nilai loss

test_loss, test_acc = model.evaluate(test_generator)

print('\nTest accuracy: ', test_acc)

In [ ]:
# menyimpan model pada epoch terakhir
# jika kalian training dengan parameter yang berbeda-beda, 
# sebaiknya dalam penamaan file tambahkan juga informasi parameter-parameter tersebut (misal nama model, epoch, input size, akurasi dll)

model.save('/content/drive/MyDrive/Landsat/Model/ResNet50 x224.h5');

In [ ]:
# menyimpan history nilai akurasi dan nilai training jika mau

import pickle
with open('/content/drive/MyDrive/Landsat/history/ResNet50 x224-history.pickle', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)